In [1]:
## IMPORT NECESSARY LIBRARIES 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import autograd.numpy as npg
from autograd import grad, elementwise_grad as e_grad

In [3]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, CrosshairTool, HoverTool, NumeralTickFormatter, Span, DatetimeTickFormatter
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot, layout
from bokeh.transform import cumsum
from bokeh.colors import RGB

In [4]:
import glob
import os
import sys
import time
from datetime import datetime,date
import math
import requests
import json
from typing import List, Tuple

In [5]:
import yfinance as yf
import bankroll
import colorsys
# import cpi

In [6]:
from scipy.optimize import minimize, minimize_scalar

## Set constants

In [7]:
num_profiles = 5000
inflation_rate = 0.06

start_date = '2000-06-01 00:00:00'
end_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

plt.rcParams['figure.figsize'] = [15, 5]
np.random.seed(42)

In [8]:
MAX = 'MAX'
MIN = 'MIN'

In [9]:
profiles_names = [
    "Roth IRA",
    "Investment Account",
    "Amazon ",
    "Cash Account",
]
profiles_targets:List[Tuple[float, float]] = [ 
    # Tuples are risk, return
    (0.10, MAX),
    None,
    (0.20, MAX),
    None
]

## Defining Functions

In [10]:
class AdamOptim():
    def __init__(self, eta=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.m_dw, self.v_dw = 0, 0
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.eta = eta
    def update(self, t, w, dw):
        self.m_dw = np.add(np.multiply(self.beta1,self.m_dw),np.multiply((1-self.beta1), dw))

        self.v_dw = np.add(np.multiply(self.beta2, self.v_dw), np.multiply((1-self.beta2),np.square(dw)))

        m_dw_corr = np.divide(self.m_dw,np.subtract((1+self.epsilon),np.power(self.beta1,t)))
        v_dw_corr = np.divide(self.v_dw,np.subtract((1+self.epsilon),np.power(self.beta2,t)))

        w = np.subtract(w, np.multiply(self.eta, np.divide(m_dw_corr, np.add(np.sqrt(v_dw_corr), self.epsilon))))
        return w

In [11]:
def norm(x):
    x = x - np.min(x)
    x = x / np.sum(x)
    return x

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def tanh_refined(x):
    return (np.tanh(x)+1.0)/2.0

def get_weights(wts):
#     wts = sigmoid(wts)
    wts = npg.maximum(0.0, wts)
    if npg.sum(wts) != 0:
        wts = wts/npg.sum(wts)
    return wts

In [12]:
def filterTickers(ticks, tick_allowed):
    return [tick for tick, allowed in zip(ticks, tick_allowed) if allowed]

def get_return(wts):
    wts = np.array(wts)
#     wts = get_weights(wts)
    port_ret = npg.sum(log_ret_mean * wts)
    port_ret = np.exp(port_ret*252) - 1
    return port_ret
    
def get_risk(wts):
    wts = np.array(wts)
#     wts = get_weights(wts)
    port_sd = npg.sqrt(npg.dot(wts.T, npg.dot(cov_mat, wts)))
    return port_sd

def get_sharpe(wts):
    port_ret = get_return(wts)
    port_sd = get_risk(wts)
    sr = port_ret / port_sd
    return sr

In [13]:
def get_weights_v(wts):
#     wts = sigmoid(wts)
    wts = npg.maximum(0.0, wts)
    if npg.sum(wts) != 0:
        wts = wts/npg.sum(wts, axis=1, keepdims=True)
    return wts

def get_return_v(wts):
    wts = get_weights_v(wts)
    port_ret = npg.sum(log_ret_mean * wts, axis=1)
    port_ret = npg.exp(npg.multiply(port_ret, 252)) - 1
    return port_ret
    
def get_risk_v(wts):
    wts = get_weights_v(wts)
    port_sd = npg.sqrt(npg.sum(wts * npg.dot(wts, cov_mat.T), axis=1))
    return port_sd

def get_sharpe_v(wts):
    port_ret = get_return_v(wts)
    port_sd = get_risk_v(wts)
    sr = port_ret / port_sd
    return sr

## For adding cash

In [14]:
def get_weights_ratio(wts_1, wts_2, ratio):
    wts_1 = np.array(wts_1)
    wts_2 = np.array(wts_2)
    wts = wts_1 * ratio + wts_2 * (1.0 - ratio)
    wts = npg.maximum(0.0, wts)
    if npg.sum(wts) != 0:
        wts = wts/npg.sum(wts)
    return wts

In [15]:
def filterTickers(ticks, tick_allowed):
    return [tick for tick, allowed in zip(ticks, tick_allowed) if allowed]

def get_return_ratio(wts_1,wts_2,ratio):
    wts = get_weights_ratio(wts_1,wts_2,ratio)
    port_ret = npg.sum(log_ret_mean * wts)
    port_ret = np.exp(port_ret*252) - 1
    return port_ret
    
def get_risk_ratio(wts_1,wts_2,ratio):
    wts = get_weights_ratio(wts_1,wts_2,ratio)
    port_sd = npg.sqrt(npg.dot(wts.T, npg.dot(cov_mat, wts)))
    return port_sd

def get_sharpe_ratio(wts_1,wts_2,ratio):
    port_ret = get_return_ratio(wts_1,wts_2,ratio)
    port_sd = get_risk_ratio(wts_1,wts_2,ratio)
    sr = port_ret / port_sd
    return sr

In [16]:
def get_gd_weights(weights_size, loss_fun, batch, iterations, LR, scorings, initial=None):
    # Optimize weights using gradient descent.
    if initial is not None:
        best_weights = get_weights(initial) + np.random.uniform(size=weights_size)
    else:
        best_weights = np.random.uniform(size=weights_size)
    training_gradient_fun = grad(loss_fun)
    scores = np.zeros((len(scorings), batch, iterations))
    for b in range(batch):
        if initial is not None:
            wts = get_weights(initial) + np.random.uniform(size=weights_size)
        else:
            wts = np.random.uniform(size=weights_size)
        for i in range(iterations):
            wts = wts - training_gradient_fun(wts) * LR
            for s in range(len(scorings)):
                scores[s, b, i] = scorings[s](wts)
        if loss_fun(wts) < loss_fun(best_weights):
            best_weights = wts
    return best_weights, scores

def calculate_weights(weights_size, loss_fun_v, batch, iterations, LR, scorings, initial=None):
    adam = AdamOptim(eta=LR)
    training_gradient_fun_v = e_grad(loss_fun_v)
    scores = np.zeros((len(scorings), batch, iterations))
    wts = np.random.uniform(size=(batch, weights_size))*4
    print(wts.shape)
    for i in range(iterations):
        dw = training_gradient_fun_v(wts)
        wts = adam.update(i+1,wts, dw)
        for s in range(len(scorings)):
            scores[s, :, i] = scorings[s](wts)
    best_weights = wts[np.argmin(loss_fun_v(wts)),:]
    return best_weights, scores

In [73]:
def plot_risk_vs_return(risk, returns, risk_title="Risk", return_title="Return"):
    fig, ax = plt.subplots(1,2)
    ax[0].plot(risk.T)
    ax[0].set_title(risk_title)
    ax[1].plot(returns.T)
    ax[1].set_title(return_title)
    
    ax[0].set_ylabel('Risk')
    ax[0].set_xlabel('Iteration')
    ax[1].set_ylabel('Return')
    ax[1].set_xlabel('Iteration')
    plt.show()

    # Portfolio composition. Min variance, max SR, max return
def plot_portfolio_composition(ticks, weights, plot_name, color_list, cash=None):
    x = dict()
    c = dict()
    for i in range(len(ticks)):
        if weights[i] >= 0.95:
            if cash:
                x[ticks[i]] = weights[i] * (cash)
                c[ticks[i]] = color_list[i]
                x[ticks[i]+" "] = weights[i] * (cash)
                c[ticks[i]+" "] = color_list[i]
            else:
                x[ticks[i]] = weights[i]
                c[ticks[i]] = color_list[i]
                x[ticks[i]+" "] = weights[i]
                c[ticks[i]+" "] = color_list[i]
        elif weights[i] > 0.001:
            if cash:
                x[ticks[i]] = weights[i] * (cash)
                c[ticks[i]] = color_list[i]
            else:
                x[ticks[i]] = weights[i]
                c[ticks[i]] = color_list[i]

    plot_data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'stock'})
    plot_data['angle'] = plot_data['value'] / plot_data['value'].sum() * 2 * math.pi
    
    plot_data['color'] = c.values()
    
    if cash:
        p = figure(width=50, height=50, title=plot_name, toolbar_location=None,sizing_mode = "scale_height",
                      tools="hover", tooltips="@stock: $@value{0,0.00} ", x_range=(-0.5,0.5))
    else:
        p = figure(width=50, height=50, title=plot_name, toolbar_location=None,sizing_mode = "scale_height",
                      tools="hover", tooltips="@stock: @value{%0.1f}", x_range=(-0.5,0.5))
    p.title.align = 'center'
    p.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                   line_color="white", color='color', source=plot_data)
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    p.outline_line_color = None

    return p

### Define Ticks

In [18]:
stocks = pd.read_csv("configs/stockslimited.csv", "\t", index_col=0, header=None).loc[:,:2]

In [19]:
stocks_limit = 100

In [80]:
ticks = [
#     "FFSFX",
    "XLRE",
    "XLV",
    "FLSW",
#     "ACLTX",
    "FCOM",
#     "TRZBX",
#     "TILWX",
#     "FITLX",
#     "FNIDX",
#     "FNDSX",
    "SUSA",
    "IQSU", 
    "USSG",
    "SUSB",
    "SNPE",
    "SUSL",
    "EAGG",
#     "WSBFX",
#     "NEXTX",
    "CASH",
]

tick_allowed = [
#     True,#"FFSFX",
    True,#"XLRE",
    True,#"XLV",
    True,#"FLSW",
#     True,#"ACLTX",
    True,#"FCOM",
#     True,#"TRZBX",
#     True,#"TILWX",
#     True,#"FITLX",
#     True,#"FNIDX",
#     True,#"FNDSX",
    True,#"SUSA",
    True,#"IQSU", 
    True,#"USSG",
    True,#"SUSB",
    True,#"SNPE",
    True,#"SUSL",
    True,#"EAGG",
#     False,#"WSBFX",
#     False,#"NEXTX",
    False,#"CASH",
]

tick_names =[ 
    yf.Ticker(ticker).info['longName'] for ticker in [
#     "FFSFX",
    "XLRE",
    "XLV",
    "FLSW",
#     "ACLTX",
    "FCOM",
        
#     "TRZBX",
#     "TILWX",
#     "FITLX",
#     "FNIDX",
#     "FNDSX",
        
    "SUSA",
    "IQSU", 
    "USSG",
    "SUSB",
    "SNPE",
    "SUSL",
    "EAGG",
        
#     "WSBFX",
#     "NEXTX",
]] + ["CASH"]

ticks_filtered = filterTickers(ticks, tick_allowed)
tick_name_filtered = filterTickers(tick_names, tick_allowed)
assert len(ticks) == len(tick_allowed)
assert len(ticks) == len(tick_names)

In [81]:
ticks

['XLRE',
 'XLV',
 'FLSW',
 'FCOM',
 'SUSA',
 'IQSU',
 'USSG',
 'SUSB',
 'SNPE',
 'SUSL',
 'EAGG',
 'CASH']

In [82]:
tick_names

['The Real Estate Select Sector SPDR Fund',
 'Health Care Select Sector SPDR Fund',
 'Franklin Templeton ETF Trust - Franklin FTSE Switzerland ETF',
 'Fidelity MSCI Communication Services Index ETF',
 'iShares Trust - iShares MSCI USA ESG Select ETF',
 'IQ Candriam ESG US Equity ETF',
 'Xtrackers MSCI USA ESG Leaders Equity ETF',
 'iShares Trust - iShares ESG Aware 1-5 Year USD Corporate Bond ETF',
 'Xtrackers S&P 500 ESG ETF',
 'iShares Trust - iShares ESG MSCI USA Leaders ETF',
 'iShares Trust - iShares ESG Aware U.S. Aggregate Bond ETF',
 'CASH']

## Define Accounts

In [50]:
path = "./configs/profiles"
FILE = open(path, 'r')
profiles = FILE.readlines()
FILE.close()
profiles = [profile.strip() for profile in profiles]

In [51]:
sys.float_info.min

2.2250738585072014e-308

## Get the API Key

In [52]:
path = "./configs/apikey"
FILE = open(path, 'r')
api_key = FILE.readline()
FILE.close()

## Get the data

In [24]:
## MAIN BODY 

# Download historical data
start = int(time.mktime(datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timetuple()))
end = int(time.mktime(datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timetuple()))

data_dict = dict()
i = 1

for stock in ticks_filtered:
    if i % 20 == 0:
        print("Sleeping for request limit")
        time.sleep(60)
    
#     querystring = {"to": end, "symbol": stock, "from": start, "resolution": 'D'}
    querystring = f"symbol={stock}&resolution=D&from={start}&to={end}&token={api_key}"

    try:
        response = requests.request("GET", url = (f"https://finnhub.io/api/v1/stock/candle?{querystring}"))
        print(f"https://finnhub.io/api/v1/stock/candle?{querystring}")
        
        data = response.json()
        df = pd.DataFrame.from_dict(data)
        df = df.drop(columns=['s', 'h', 'l', 'o', 'v'])

        # Output time zone: Universal Time Coordinated
        df['time'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in df.t.values]

    except:
        raise ValueError("No data found for " + stock)

    i += 1
    print(i)

    # The download limit is 10 requests per minute
            
    data_dict[stock] = df




https://finnhub.io/api/v1/stock/candle?symbol=FFSFX&resolution=D&from=959832000&to=1641216781&token=c6g2jhaad3id0r6gcvl0
2
https://finnhub.io/api/v1/stock/candle?symbol=ACLTX&resolution=D&from=959832000&to=1641216781&token=c6g2jhaad3id0r6gcvl0
3
https://finnhub.io/api/v1/stock/candle?symbol=TRZBX&resolution=D&from=959832000&to=1641216781&token=c6g2jhaad3id0r6gcvl0
4
https://finnhub.io/api/v1/stock/candle?symbol=TILWX&resolution=D&from=959832000&to=1641216781&token=c6g2jhaad3id0r6gcvl0
5
https://finnhub.io/api/v1/stock/candle?symbol=FITLX&resolution=D&from=959832000&to=1641216781&token=c6g2jhaad3id0r6gcvl0
6
https://finnhub.io/api/v1/stock/candle?symbol=FNIDX&resolution=D&from=959832000&to=1641216781&token=c6g2jhaad3id0r6gcvl0
7
https://finnhub.io/api/v1/stock/candle?symbol=FNDSX&resolution=D&from=959832000&to=1641216781&token=c6g2jhaad3id0r6gcvl0
8


In [25]:
# PRE PROCESS DATA TO A FRIENDLY FORMAT
data_to_concat = []
data_dict_new = {}
for key in data_dict:
    data_dict_new[key] = data_dict[key].rename(columns={"c": key})
    data_dict_new[key]['time'] = pd.to_datetime(data_dict_new[key]['time']).dt.date
    data_dict_new[key] = data_dict_new[key].set_index("time")
    data_to_concat.append(data_dict_new[key])
    
price_data = pd.concat(data_to_concat, axis=1)
price_data = price_data.loc[:,~price_data.columns.duplicated()].drop(columns='t')
price_data = price_data.sort_index()

log_ret = np.log(price_data/price_data.shift(1))

cov_mat = np.array(np.exp(log_ret.cov()*252)-1)
log_ret_mean = np.array(log_ret.mean())


In [26]:
price_data

,FFSFX,ACLTX,TRZBX,TILWX,FITLX,FNIDX,FNDSX
2006-05-15,NaN,9.98,NaN,NaN,NaN,NaN,NaN
2006-05-16,NaN,9.97,NaN,NaN,NaN,NaN,NaN
2006-05-17,NaN,9.80,NaN,NaN,NaN,NaN,NaN
2006-05-18,NaN,9.71,NaN,NaN,NaN,NaN,NaN
2006-05-19,NaN,9.75,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2021-12-27,14.09,24.62,180.86,24.34,21.31,13.18,10.69
2021-12-28,14.08,24.50,179.82,24.20,21.28,13.19,10.69
2021-12-29,14.08,24.49,179.32,24.18,21.33,13.19,10.66
2021-12-30,13.38,24.43,179.24,24.16,21.26,13.22,10.68


In [27]:
cov_mat.diagonal()

array([0.04052935, 0.0507208 , 0.04892488, 0.11489245, 0.04135304,
       0.02973361, 0.00141597])

In [28]:
# price_data["ASML"][-365:]

## Load Profile Data

In [53]:
list_of_files = glob.glob('.\profiles\*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)
df = pd.read_csv(latest_file)
df = df[~df.Description.isna() | (df.Symbol == "Pending Activity")]
df = df[['Account Number', 'Symbol', 'Current Value']]

.\profiles\Portfolio_Positions_Jan-03-2022.csv


In [54]:
profile_makeup = pd.DataFrame(data = np.zeros((len(profiles), len(ticks))), 
                              index=profiles, 
                              columns=ticks)
profile_makeup

,FFSFX,ACLTX,TRZBX,TILWX,FITLX,FNIDX,FNDSX,WSBFX,NEXTX,CASH
***REMOVED***,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
***REMOVED***,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
***REMOVED***,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
for row_ in df.iterrows():
    if row_[1]["Symbol"] in ticks and  row_[1]["Account Number"] in profiles:
        profile_makeup.loc[row_[1]["Account Number"], row_[1]["Symbol"]] = float(row_[1]["Current Value"].replace("$",""))
    elif row_[1]["Account Number"] in profiles:
        profile_makeup.loc[row_[1]["Account Number"], "CASH"] += float(row_[1]["Current Value"].replace("$",""))
profile_makeup

,FFSFX,ACLTX,TRZBX,TILWX,FITLX,FNIDX,FNDSX,WSBFX,NEXTX,CASH
***REMOVED***,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,1131.77
***REMOVED***,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.01
***REMOVED***,0.0,0.0,0.0,0.0,9280.89,0.0,159.58,0.0,0.0,0.00


## Get Inflation Data

In [56]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0'],"startyear":"2020", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)


In [57]:
df = pd.DataFrame.from_dict(json_data['Results']['series'][0]['data'])
df.head()
values = df["value"].to_numpy().astype(float)
inflations = (values[:-12] - values[12:])/values[12:]  
current_inflation = inflations[0]
avg_inflation = inflations.mean()

## Get Limits

### Get Max Sharpe

In [58]:
loss = lambda x: -get_sharpe(x)
best_sharpe_weights = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    ).x
get_sharpe(best_sharpe_weights)

1.0800855194836447

### Get Risk Limits

In [59]:
loss = get_risk
min_risk_weights = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    ).x
get_risk(min_risk_weights)

0.03464361628251537

In [60]:
loss = lambda x: -get_risk(x)
max_risk_weights = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    ).x
get_risk(max_risk_weights)

0.3389578885955929

### Get Return Range

In [61]:
loss = lambda x: get_return(x)
min_return_weights = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    ).x
get_return(min_return_weights)

0.004533937733845361

In [62]:
loss = lambda x: -get_return(x)
rts = minimize(
      loss,
      np.random.random(len(ticks_filtered)),
      constraints=[
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
      ],
      bounds=[(0., 1.) for i in range(len(ticks_filtered))]
    )
max_return_weights = rts.x
get_return(max_return_weights)

0.1755137142115697

## Get List of Max Returns for Spread of risks

In [63]:
risk_min = get_risk(min_risk_weights)
risk_max = get_risk(max_return_weights)
risks_count = 50
risks_bot = np.linspace(risk_min, risk_max, risks_count, True)

### Max Sharpe Spreads

In [64]:
best_weights_range = np.random.random(size=(risks_count, len(ticks_filtered))) 
start_time = time.time()
for r in range(risks_count):
    loss = lambda x: -get_return(x)
    rts = minimize(
          loss,
          np.random.random(len(ticks_filtered)),
          constraints=[
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
            {'type': 'ineq', 'fun': lambda w, risk=risks_bot[r]: -(get_risk(w) - risk)},
          ],
          bounds=[(0., 1.) for i in range(len(ticks_filtered))]
        )
    best_weights_range[r, :] = rts.x

    print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00899815559387207 seconds ---
--- 0.013000249862670898 seconds ---
--- 0.017998933792114258 seconds ---
--- 0.020999431610107422 seconds ---
--- 0.023999452590942383 seconds ---
--- 0.027998924255371094 seconds ---
--- 0.030999183654785156 seconds ---
--- 0.03499913215637207 seconds ---
--- 0.038999080657958984 seconds ---
--- 0.042999267578125 seconds ---
--- 0.046999454498291016 seconds ---
--- 0.05199933052062988 seconds ---
--- 0.054999351501464844 seconds ---
--- 0.05899930000305176 seconds ---
--- 0.06199908256530762 seconds ---
--- 0.06499958038330078 seconds ---
--- 0.0689992904663086 seconds ---
--- 0.07199907302856445 seconds ---
--- 0.07599949836730957 seconds ---
--- 0.07899928092956543 seconds ---
--- 0.0839986801147461 seconds ---
--- 0.08799910545349121 seconds ---
--- 0.09099841117858887 seconds ---
--- 0.09499955177307129 seconds ---
--- 0.0989997386932373 seconds ---
--- 0.10299921035766602 seconds ---
--- 0.10699915885925293 seconds ---
--- 0.10999917984008789 

## Build Profiles

In [65]:
profiles_constraints = []
profiles_losses = []
for target in profiles_targets:
    if target is None:
        profiles_constraints.append(None)
        profiles_losses.append(None)
    else:
        risk_t, return_t = target
        print(risk_t, return_t)
        if (risk_t == MAX or risk_t == MIN) and (return_t == MAX or return_t == MIN):
            profiles_constraints.append(None)
            if (risk_t == MAX) and (return_t == MAX):
                profiles_losses.append(lambda x:-(get_return(x) * get_risk(x)))
            elif (risk_t == MAX) and (return_t == MIN):
                profiles_losses.append(lambda x: get_sharpe(x))
            elif (risk_t == MIN) and (return_t == MAX):
                profiles_losses.append(lambda x: -get_sharpe(x))
            elif (risk_t == MIN) and (return_t == MIN):
                profiles_losses.append(lambda x: get_return(x) * get_risk(x))
        elif (risk_t == MAX or risk_t == MIN):
            print(return_t)
            profiles_constraints.append({'type': 'eq', 'fun': lambda x, return_t=return_t: (get_return(x) - return_t)}),
            if risk_t == MAX:
                profiles_losses.append(lambda x: -get_risk(x))
            elif risk_t == MIN:
                profiles_losses.append(lambda x: get_risk(x))
        elif (return_t == MAX or return_t == MIN):
            print(risk_t)
            profiles_constraints.append({'type': 'eq', 'fun': lambda x, risk_t=risk_t: (get_risk(x) - risk_t)}),
            if return_t == MAX:
                profiles_losses.append(lambda x: -get_return(x))
            elif return_t == MIN:
                profiles_losses.append(lambda x: get_return(x))   
        else:
            profiles_constraints.append(None)
            profiles_losses.append(lambda x,
                                   return_t=return_t,
                                   risk_t=risk_t: 
                                   np.abs(risk_t - get_risk(x)) * np.abs(return_t - get_return(x)))

0.1 MAX
0.1
0.2 MAX
0.2


In [66]:
b = 1000
i = 1000
lr = 0.015
batch = [b]*len(profiles)
iterations = [i]*len(profiles)
LR=[lr]*len(profiles)

LR[2]=0.005
target_weights = []

for i in range(len(profiles)):
    print(i)
    if profiles_targets[i] is not None:
        
        loss = profiles_losses[i]
        start_time = time.time()
        print(profiles_constraints[i])
        rts = minimize(
              loss,
              np.random.random(len(ticks_filtered)),
              constraints=[
                {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},
                profiles_constraints[i],
              ] if profiles_constraints[i] is not None else [
                {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.},],
              bounds=[(0., 1.) for i in range(len(ticks_filtered))]
            )

        print(rts.success)
        target_weights.append(np.concatenate([rts.x, [0,0,0]]))
        print(r)
        if profiles_constraints[i] is not None:
            print(0.1)
        print(get_risk(rts.x))
        print(get_return(rts.x))

        print("--- %s seconds ---" % (time.time() - start_time))
        
    else:
        
        wts = profile_makeup.loc[profiles[i]].to_numpy()
        target_weights.append(wts)


0
{'type': 'eq', 'fun': <function <lambda> at 0x000001A7F06F4488>}
True
49
0.1
0.10000000000310492
0.09502824687194877
--- 0.005000114440917969 seconds ---
1
2
{'type': 'eq', 'fun': <function <lambda> at 0x000001A7F06F4B70>}
True
49
0.1
0.20000002319550503
0.17285298216362066
--- 0.00500035285949707 seconds ---


## Build Buys

## Rebalancing

In [67]:
profile_changes = []
for i in range(len(profiles)):
    print("=====================")
    profile_sum = np.sum(profile_makeup.loc[profiles[i]].to_numpy())
    print(profile_sum)
    target_sum = np.sum(target_weights[i])
    print(target_sum)
    changes = target_weights[i] * (profile_sum/target_sum) - profile_makeup.loc[profiles[i]].to_numpy()
    print(changes)
    profile_changes.append(changes)

1131.77
1.0
[ 0.00000000e+00  6.13381451e-14  2.76256141e-13  0.00000000e+00
  5.70821926e+02  7.96955062e-14  5.60948074e+02  0.00000000e+00
  0.00000000e+00 -1.13177000e+03]
0.01
0.01
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
9440.47
1.000000000000001
[ 0.00000000e+00  1.01424194e-12  3.63654580e-12  0.00000000e+00
  1.00860647e+01  4.19949160e-12 -1.00860647e+01  0.00000000e+00
  0.00000000e+00  0.00000000e+00]


## Display Graphs

In [68]:
SAT = 0.5
LUM = 0.5

In [69]:
color_list = [colorsys.hls_to_rgb(h, LUM, SAT) for h in np.linspace(0.0, 1.0, len(ticks), endpoint=False)]
color_list = [RGB(r*255,g*255,d*255) for r,g,d in color_list]
color_list_accounts = [colorsys.hls_to_rgb(h, LUM, SAT) for h in np.linspace(0.0, 1.0, len(profiles), endpoint=False)]
color_list_accounts = [RGB(r*255,g*255,d*255) for r,g,d in color_list_accounts]

# Plotting

In [70]:
SAT = 0.5
LUM = 0.5

In [71]:
color_list = [colorsys.hls_to_rgb(h, LUM, SAT) for h in np.linspace(0.0, 1.0, len(ticks), endpoint=False)]
color_list = [RGB(r*255,g*255,d*255) for r,g,d in color_list]
color_list_accounts = [colorsys.hls_to_rgb(h, LUM, SAT) for h in np.linspace(0.0, 1.0, len(profiles), endpoint=False)]
color_list_accounts = [RGB(r*255,g*255,d*255) for r,g,d in color_list_accounts]

In [74]:
source = ColumnDataSource(price_data)
# ===== Setup Plot ====
p = figure(
    sizing_mode = "stretch_both", 
    title="Efficient frontier.",
    tools='box_zoom,wheel_zoom,reset', 
    toolbar_location='right',
    x_range=(-0.25,1.0)
)
p.add_tools(CrosshairTool(line_alpha=1, line_color='lightgray', line_width=1))
p.add_tools(HoverTool(tooltips=None))

p.xaxis.axis_label = 'Volatility, or risk (standard deviation)'
p.yaxis.axis_label = 'Annual return'
p.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")
# ===== Render Boundries ====
risk_boundry = Span(location=get_risk(min_risk_weights),
                    dimension='height', line_color='#3A5311',
                    line_width=1)
return_boundry = Span(location=get_return(max_return_weights), 
                      dimension='width', line_color='#3A5311',
                      line_width=1)
current_inf_boundry = Span(location=current_inflation, 
                      dimension='width', line_color='#03C04A',
                           line_width=1)
average_inf_boundry = Span(location=avg_inflation, 
                      dimension='width', line_color='#607D3B',
                           line_width=1)
p.renderers.extend([risk_boundry, return_boundry, current_inf_boundry, average_inf_boundry])
# ===== Render Best Sharpe Line ====
boundry =np.concatenate([
    best_weights_range, 
])

l = p.line(
    get_risk_v(boundry),
    get_return_v(boundry), 
    color="purple",
#     legend_label="Max Sharpe Line?",
    line_width=1)

p.add_tools(HoverTool(renderers=[l], tooltips=[
    ('Name', "Max Sharpe Line")
]))
# ===== Render Sharpe Lines ====
p.line([0,get_return(max_return_weights)],
       [0,get_return(max_return_weights)],
#        legend_label="Sharpe Of 1",
       color="#00B7EB",line_width=1)
p.line([0,0.5*get_return(max_return_weights)],
       [0,get_return(max_return_weights)],
#        legend_label="Sharpe Of 2",
       color="#6495ED",line_width=1)
p.line([0,(1.0/3.0)*get_return(max_return_weights)],
       [0,get_return(max_return_weights)],
#        legend_label="Sharpe Of 3",
       color="#007FFF",line_width=1)
# ====== Render Prices ======
t = figure(
    sizing_mode = "stretch_both", 
    title="Prices Over Time",
    tools='box_zoom,wheel_zoom,reset', 
    toolbar_location='right',
    x_axis_type='datetime'
#     x_range=(-0.25,1.0)
)
# timestamp_start = (datetime.combine(datepicker_start.value, datetime.min.time())
#                         - datetime(1970, 1, 1)) / timedelta(seconds=1)
# timestamp_end = (datetime.combine(datepicker_end.value, datetime.min.time())
#                     - datetime(1970, 1, 1)) / timedelta(seconds=1)
t.x_range.start = (datetime.now().timestamp() - 720*3600) * 1000  # Multiply by 1e3 as JS timestamp is in milliseconds
t.x_range.end   = datetime.now().timestamp() * 1000  # Multiply by 1e3 as JS timestamp is in milliseconds


t.add_tools(CrosshairTool(line_alpha=1, line_color='lightgray', line_width=1))
t.add_tools(HoverTool(tooltips=None))

t.xaxis.axis_label = 'Date'
t.yaxis.axis_label = 'Price'
t.xaxis[0].formatter = DatetimeTickFormatter(days=["%b %d, %Y"])
t.yaxis[0].formatter = NumeralTickFormatter(format="$0.0")

# ===== Plot prices =====
range_ = 365*5
source = ColumnDataSource((price_data-price_data[-range_:].mean())/price_data[-range_:].std())
for i in range(len(ticks_filtered)):
    t.line(x = "index", y=ticks_filtered[i],
             source=source,
             legend_label=ticks_filtered[i], 
             name=ticks_filtered[i], 
             color=color_list[i])

# ===== Render Rebalance Buy Charts ====
fidelity_buy_pies = []
for i in range(len(profiles)):
    
    if profiles_targets[i] is not None:
        wts = get_weights(profile_changes[i] * (profile_changes[i] > 0))
    else:
        wts = get_weights(profile_makeup.loc[profiles[i]].to_numpy())
    
    cash = np.sum(profile_changes[i] * (profile_changes[i] > 0))

    fidelity_buy_pies.append(
        plot_portfolio_composition(ticks,
                                   wts,
                                   profiles_names[i] + " Buy",
                                   color_list,
                                   cash = cash))
#     wts_filtered = filterTickers(wts_point, tick_allowed)

print(fidelity_buy_pies)
# ===== Render Rebalance Sell Pie Charts ====
fidelity_sell_pies = []
for i in range(len(profiles)):
    
    if profiles_targets[i] is not None:
        wts = get_weights(-profile_changes[i] * (profile_changes[i] < 0))
        
    else:
        wts = get_weights(profile_makeup.loc[profiles[i]].to_numpy())
    

    cash = np.sum(-profile_changes[i] * (profile_changes[i] < 0))
                            
    fidelity_sell_pies.append(
        plot_portfolio_composition(ticks,
                                   wts,
                                   profiles_names[i] + " Sell",
                                   color_list,
                                   cash = cash))
#     wts_filtered = filterTickers(wts_point, tick_allowed)

print(fidelity_sell_pies)
# ===== Render Target Profile Pie Charts ====
fidelity_targets = []
renderers = []

for i in range(len(target_weights)):
    fidelity_targets.append(
        plot_portfolio_composition(
            (ticks if len(target_weights[i]) > len(ticks_filtered) else ticks_filtered),
            get_weights(target_weights[i]),
            profiles_names[i] + " Target",
            color_list
        ))
    wts_filtered = filterTickers(target_weights[i], tick_allowed)
    if np.sum(wts_filtered) != 0.0:
        c = p.circle(get_risk(get_weights(wts_filtered)),
                     get_return(get_weights(wts_filtered)),
                     color=color_list_accounts[i],
                     alpha=0.6,
                     name=profiles_names[i] + " Target",
                     legend_label=profiles_names[i] + " Target",
                     size=15)
        renderers.append(c)
# ===== Render Existing Profile Pie Charts ====
fidelity_pies = []
tooltips = []
for i in range(len(profiles)):
    wts = get_weights(profile_makeup.loc[profiles[i]].to_numpy())

    fidelity_pies.append(
        plot_portfolio_composition(ticks,
                                   wts,
                                   profiles_names[i],
                                   color_list))
    wts_filtered = filterTickers(wts, tick_allowed)
    if np.sum(wts_filtered) != 0.0:
        c = p.circle(get_risk(wts_filtered), 
                     get_return(wts_filtered), 
                     color=color_list_accounts[i], 
                     name=profiles_names[i],
                     legend_label=profiles_names[i],
                     size=15)
        renderers.append(c)        

tooltips.append(('Profile', "$name"))
p.add_tools(HoverTool(renderers=renderers, tooltips=tooltips))
# ===== Render Funds ====
wts = np.eye(len(ticks_filtered))
risks_ = get_risk_v(wts)
returns_ = get_return_v(wts)
colors = filterTickers(color_list, tick_allowed)
renderers = []
for i in range(len(ticks_filtered)):
    c = p.circle(risks_[i],returns_[i],
             color=color_list[i],
             legend_label=ticks_filtered[i], 
             name=ticks_filtered[i], 
             size=10, alpha=0.8, )
    renderers.append(c)
p.add_tools(HoverTool(renderers=renderers, tooltips=[
    ('Tick', "$name")
]))

# ===== Adjusting Legend ====
p.legend.location = "top_left"
# p.legend.visible = False
p.legend.click_policy="hide"
p.legend.__setattr__('label_text_font_size', '8pt')


t.legend.location = "top_left"
# t.legend.visible = False
t.legend.click_policy="hide"
t.legend.__setattr__('label_text_font_size', '8pt')
# ===== Create dashboard and open new window to show results ====
layout_ = row([ 
        column([
            p,
            t,
        ], sizing_mode = "stretch_both"),
    column([
        row(fidelity_pies,
               sizing_mode = "stretch_height"), 
        row(fidelity_targets
               , sizing_mode = "stretch_height"),
        row(fidelity_sell_pies
               , sizing_mode = "stretch_height"),
        row(fidelity_buy_pies
               , sizing_mode = "stretch_height"),
    ])
    ],width = 1350, sizing_mode = "stretch_height")
    
show(layout_)

[Figure(id='6930', ...), Figure(id='6960', ...), Figure(id='6990', ...)]
[Figure(id='7020', ...), Figure(id='7050', ...), Figure(id='7080', ...)]
